In [1]:
import pypsa
import pandas as pd
import numpy as np
import time
import cufflinks as cf
import plotly.offline as pltly
pltly.init_notebook_mode(connected=True)
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

C:\Users\Tim\Anaconda3\lib\site-packages\dask\config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}
C:\Users\Tim\Anaconda3\lib\site-packages\distributed\config.py:20: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  defaults = yaml.load(f)


### Loading network from file

In [2]:
network = pypsa.Network()

In [3]:
network.import_from_hdf5('Scandinavia')

INFO:pypsa.io:Imported network Scandinavia has buses, generators, lines, loads


In [4]:
network.import_from_hdf5('euro-30')
network.snapshots = network.snapshots[0:2]
network.snapshots

INFO:pypsa.io:Imported network euro-30 has buses, carriers, generators, links, loads


DatetimeIndex(['2011-01-01 00:00:00', '2011-01-01 01:00:00'], dtype='datetime64[ns]', name='name', freq='H')

### Initial solution of network

In [5]:
solver_options = {
  'method': 2,
  'threads': 8,
  'logfile': 'solver.log',
  'BarConvTol' : 1.e-8, #1.e-12 ###1e-8 # [0..1]
  'crossover' : 0,   # or -1
  'FeasibilityTol' : 1.e-6 }#1e-2 ###1e-6 # [1e-9..1e-2]
network.lopf(network.snapshots, solver_name='gurobi',solver_options=solver_options)
old_objective_value = network.model.objective()

INFO:pypsa.pf:Slack bus for sub-network 0 is My bus 0
INFO:pypsa.pf:Slack bus for sub-network 1 is AT
INFO:pypsa.pf:Slack bus for sub-network 2 is BA
INFO:pypsa.pf:Slack bus for sub-network 3 is BE
INFO:pypsa.pf:Slack bus for sub-network 4 is BG
INFO:pypsa.pf:Slack bus for sub-network 5 is CH
INFO:pypsa.pf:Slack bus for sub-network 6 is CZ
INFO:pypsa.pf:Slack bus for sub-network 7 is DE
INFO:pypsa.pf:Slack bus for sub-network 8 is DK
INFO:pypsa.pf:Slack bus for sub-network 9 is EE
INFO:pypsa.pf:Slack bus for sub-network 10 is ES
INFO:pypsa.pf:Slack bus for sub-network 11 is FI
INFO:pypsa.pf:Slack bus for sub-network 12 is FR
INFO:pypsa.pf:Slack bus for sub-network 13 is GB
INFO:pypsa.pf:Slack bus for sub-network 14 is GR
INFO:pypsa.pf:Slack bus for sub-network 15 is HR
INFO:pypsa.pf:Slack bus for sub-network 16 is HU
INFO:pypsa.pf:Slack bus for sub-network 17 is IE
INFO:pypsa.pf:Slack bus for sub-network 18 is IT
INFO:pypsa.pf:Slack bus for sub-network 19 is LT
INFO:pypsa.pf:Slack bus 

# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: x655
  Lower bound: 16421323022.048317
  Upper bound: 16421323022.048317
  Number of objectives: 1
  Number of constraints: 919
  Number of variables: 655
  Number of binary variables: 0
  Number of integer variables: 0
  Number of continuous variables: 655
  Number of nonzeros: 1807
  Sense: minimize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  Return code: 0
  Message: Model was solved to optimality (subject to tolerances), and an optimal solution is available.
  Termination condition: optimal
  Termination message: Model

In [6]:
original_solution = [sum(network.generators[network.generators.type=='convetional'].p_nom_opt),
                     sum(network.generators[network.generators.type=='wind'].p_nom_opt),
                     sum(network.generators[network.generators.type=='solar'].p_nom_opt)]

In [7]:
original_solution
df_points = pd.DataFrame(columns=['type','feasable','ocgt','wind','solar'])
df_points.loc[0] = ['original',True] + original_solution

df_detail = pd.DataFrame(columns=network.generators.p_nom_opt.index)
df_detail.loc[0] = list(network.generators.p_nom_opt.values)

### Finding maximum and minimum points

In [8]:
import pyomo.environ as pyomo_env
# Defining exstra functionality, that updates the objective function of the network
def maxmin_functionality(network, snapshots,  MGA_slack = 0.05, counter = 0):
# Identify the nonzero decision variables that should enter the MGA objective function.

    maxmin_objective = 0
    variables = [gen_p for gen_p in network.model.generator_p_nom]

    types = ['OCGT','wind','olar']
    gen_p_type = [gen_p  for gen_p in variables if gen_p[-4:]==types[int(counter/2)]]
    maxmin_objective += sum([network.model.generator_p_nom[gen_p] for gen_p in gen_p_type])

    if counter%2 == 0 :
        maxmin_objective = -maxmin_objective
    print(maxmin_objective)
    # Add the new MGA objective function to the model.
    network.model.mga_objective = pyomo_env.Objective(expr=maxmin_objective)
    # Deactivate the old objective function and activate the MGA objective function.
    network.model.objective.deactivate()
    network.model.mga_objective.activate()
    # Add the MGA slack constraint.
    #print('old objective value ',old_objective_value)
    network.model.mga_constraint = pyomo_env.Constraint(expr=network.model.objective.expr <= 
                                          (1 + MGA_slack) * old_objective_value)

In [9]:

MGA_slack = 0.1


idx_l = ['max ocgt','min ocgt','max wind','min wind','max solar','min solar']

for i,idx in zip(range(6),idx_l):
    network.lopf(network.snapshots,\
                         solver_name='gurobi',\
                         extra_functionality=lambda network,\
                         snapshots: maxmin_functionality(network, snapshots, MGA_slack,i),
                         solver_options=solver_options)
    
    df_points.loc[df_points.index.max()+1]=[idx,True,
                                            sum(network.generators[network.generators.type=='convetional'].p_nom_opt),
                                            sum(network.generators[network.generators.type=='wind'].p_nom_opt),
                                            sum(network.generators[network.generators.type=='solar'].p_nom_opt)]
    df_detail.loc[df_detail.index.max()+1]=list(network.generators.p_nom_opt.values)
    

INFO:pypsa.pf:Slack bus for sub-network 0 is My bus 0
INFO:pypsa.pf:Slack bus for sub-network 1 is AT
INFO:pypsa.pf:Slack bus for sub-network 2 is BA
INFO:pypsa.pf:Slack bus for sub-network 3 is BE
INFO:pypsa.pf:Slack bus for sub-network 4 is BG
INFO:pypsa.pf:Slack bus for sub-network 5 is CH
INFO:pypsa.pf:Slack bus for sub-network 6 is CZ
INFO:pypsa.pf:Slack bus for sub-network 7 is DE
INFO:pypsa.pf:Slack bus for sub-network 8 is DK
INFO:pypsa.pf:Slack bus for sub-network 9 is EE
INFO:pypsa.pf:Slack bus for sub-network 10 is ES
INFO:pypsa.pf:Slack bus for sub-network 11 is FI
INFO:pypsa.pf:Slack bus for sub-network 12 is FR
INFO:pypsa.pf:Slack bus for sub-network 13 is GB
INFO:pypsa.pf:Slack bus for sub-network 14 is GR
INFO:pypsa.pf:Slack bus for sub-network 15 is HR
INFO:pypsa.pf:Slack bus for sub-network 16 is HU
INFO:pypsa.pf:Slack bus for sub-network 17 is IE
INFO:pypsa.pf:Slack bus for sub-network 18 is IT
INFO:pypsa.pf:Slack bus for sub-network 19 is LT
INFO:pypsa.pf:Slack bus 

- (generator_p_nom[AT OCGT] + generator_p_nom[BA OCGT] + generator_p_nom[BE OCGT] + generator_p_nom[BG OCGT] + generator_p_nom[CH OCGT] + generator_p_nom[CZ OCGT] + generator_p_nom[DE OCGT] + generator_p_nom[DK OCGT] + generator_p_nom[EE OCGT] + generator_p_nom[ES OCGT] + generator_p_nom[FI OCGT] + generator_p_nom[FR OCGT] + generator_p_nom[GB OCGT] + generator_p_nom[GR OCGT] + generator_p_nom[HR OCGT] + generator_p_nom[HU OCGT] + generator_p_nom[IE OCGT] + generator_p_nom[IT OCGT] + generator_p_nom[LT OCGT] + generator_p_nom[LU OCGT] + generator_p_nom[LV OCGT] + generator_p_nom[NL OCGT] + generator_p_nom[NO OCGT] + generator_p_nom[PL OCGT] + generator_p_nom[PT OCGT] + generator_p_nom[RO OCGT] + generator_p_nom[RS OCGT] + generator_p_nom[SE OCGT] + generator_p_nom[SI OCGT] + generator_p_nom[SK OCGT])


INFO:pypsa.opf:Optimization successful


# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: x655
  Lower bound: -381561.78331550426
  Upper bound: -381561.78331550426
  Number of objectives: 1
  Number of constraints: 920
  Number of variables: 655
  Number of binary variables: 0
  Number of integer variables: 0
  Number of continuous variables: 655
  Number of nonzeros: 2273
  Sense: minimize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  Return code: 0
  Message: Model was solved to optimality (subject to tolerances), and an optimal solution is available.
  Termination condition: optimal
  Termination message: Mod

INFO:pypsa.pf:Slack bus for sub-network 0 is My bus 0
INFO:pypsa.pf:Slack bus for sub-network 1 is AT
INFO:pypsa.pf:Slack bus for sub-network 2 is BA
INFO:pypsa.pf:Slack bus for sub-network 3 is BE
INFO:pypsa.pf:Slack bus for sub-network 4 is BG
INFO:pypsa.pf:Slack bus for sub-network 5 is CH
INFO:pypsa.pf:Slack bus for sub-network 6 is CZ
INFO:pypsa.pf:Slack bus for sub-network 7 is DE
INFO:pypsa.pf:Slack bus for sub-network 8 is DK
INFO:pypsa.pf:Slack bus for sub-network 9 is EE
INFO:pypsa.pf:Slack bus for sub-network 10 is ES
INFO:pypsa.pf:Slack bus for sub-network 11 is FI
INFO:pypsa.pf:Slack bus for sub-network 12 is FR
INFO:pypsa.pf:Slack bus for sub-network 13 is GB
INFO:pypsa.pf:Slack bus for sub-network 14 is GR
INFO:pypsa.pf:Slack bus for sub-network 15 is HR
INFO:pypsa.pf:Slack bus for sub-network 16 is HU
INFO:pypsa.pf:Slack bus for sub-network 17 is IE
INFO:pypsa.pf:Slack bus for sub-network 18 is IT
INFO:pypsa.pf:Slack bus for sub-network 19 is LT
INFO:pypsa.pf:Slack bus 

...

# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: x655
  Lower bound: -21460.289910555424
  Upper bound: -21460.289910555424
  Number of objectives: 1
  Number of constraints: 920
  Number of variables: 655
  Number of binary variables: 0
  Number of integer variables: 0
  Number of continuous variables: 655
  Number of nonzeros: 2273
  Sense: minimize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  Return code: 0
  Message: Model was solved to optimality (subject to tolerances), and an optimal solution is available.
  Termination condition: optimal
  Termination message: Mod

INFO:pypsa.pf:Slack bus for sub-network 0 is My bus 0
INFO:pypsa.pf:Slack bus for sub-network 1 is AT
INFO:pypsa.pf:Slack bus for sub-network 2 is BA
INFO:pypsa.pf:Slack bus for sub-network 3 is BE
INFO:pypsa.pf:Slack bus for sub-network 4 is BG
INFO:pypsa.pf:Slack bus for sub-network 5 is CH
INFO:pypsa.pf:Slack bus for sub-network 6 is CZ
INFO:pypsa.pf:Slack bus for sub-network 7 is DE
INFO:pypsa.pf:Slack bus for sub-network 8 is DK
INFO:pypsa.pf:Slack bus for sub-network 9 is EE
INFO:pypsa.pf:Slack bus for sub-network 10 is ES
INFO:pypsa.pf:Slack bus for sub-network 11 is FI
INFO:pypsa.pf:Slack bus for sub-network 12 is FR
INFO:pypsa.pf:Slack bus for sub-network 13 is GB
INFO:pypsa.pf:Slack bus for sub-network 14 is GR
INFO:pypsa.pf:Slack bus for sub-network 15 is HR
INFO:pypsa.pf:Slack bus for sub-network 16 is HU
INFO:pypsa.pf:Slack bus for sub-network 17 is IE
INFO:pypsa.pf:Slack bus for sub-network 18 is IT
INFO:pypsa.pf:Slack bus for sub-network 19 is LT
INFO:pypsa.pf:Slack bus 

generator_p_nom[AT solar] + generator_p_nom[BA solar] + generator_p_nom[BE solar] + generator_p_nom[BG solar] + generator_p_nom[CH solar] + generator_p_nom[CZ solar] + generator_p_nom[DE solar] + generator_p_nom[DK solar] + generator_p_nom[EE solar] + generator_p_nom[ES solar] + generator_p_nom[FI solar] + generator_p_nom[FR solar] + generator_p_nom[GB solar] + generator_p_nom[GR solar] + generator_p_nom[HR solar] + generator_p_nom[HU solar] + generator_p_nom[IE solar] + generator_p_nom[IT solar] + generator_p_nom[LT solar] + generator_p_nom[LU solar] + generator_p_nom[LV solar] + generator_p_nom[NL solar] + generator_p_nom[NO solar] + generator_p_nom[PL solar] + generator_p_nom[PT solar] + generator_p_nom[RO solar] + generator_p_nom[RS solar] + generator_p_nom[SE solar] + generator_p_nom[SI solar] + generator_p_nom[SK solar]


INFO:pypsa.opf:Optimization successful


# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: x655
  Lower bound: 0.0
  Upper bound: 0.0
  Number of objectives: 1
  Number of constraints: 920
  Number of variables: 655
  Number of binary variables: 0
  Number of integer variables: 0
  Number of continuous variables: 655
  Number of nonzeros: 2273
  Sense: minimize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  Return code: 0
  Message: Model was solved to optimality (subject to tolerances), and an optimal solution is available.
  Termination condition: optimal
  Termination message: Model was solved to optimality (sub

### Plot of max and min points 

In [10]:
%matplotlib widget
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
  
ax.plot(df_points['ocgt'],df_points['wind'],df_points['solar'], "go")

ax.set_xlabel('gas')
ax.set_ylabel('wind')
ax.set_zlabel('solar')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0.5, 0, 'solar')

### Defining functions to test individual points 

In [11]:
def in_hull(p, hull):
    """
    Test if points in `p` are in `hull`

    `p` should be a `NxK` coordinates of `N` points in `K` dimensions
    `hull` is either a scipy.spatial.Delaunay object or the `MxK` array of the 
    coordinates of `M` points in `K`dimensions for which Delaunay triangulation
    will be computed
    """
    from scipy.spatial import Delaunay
    if not isinstance(hull,Delaunay):
        hull = Delaunay(hull)

    return hull.find_simplex(p)>=0

In [12]:
def hull_centroid(hull):
    
    x = np.mean(hull.points[:,0])
    y = np.mean(hull.points[:,1])
    z = np.mean(hull.points[:,2])
    centroid = [x,y,z]
    return centroid 

In [13]:
def fill_hull(df_points,hull,dens):


    h_min = hull.min_bound
    h_max = hull.max_bound
    x,y,z = np.meshgrid(np.linspace(h_min[0],h_max[0],dens),np.linspace(h_min[1],h_max[1],dens),np.linspace(h_min[2],h_max[2],dens))
    x = x.reshape((dens)**3)
    y = y.reshape((dens)**3)
    z = z.reshape((dens)**3)
 
    grid = [[x_,y_,z_] for x_,y_,z_ in zip(x,y,z)]
    new_points = [point for point in grid if all(in_hull([point],hull))]
    new_points = np.array(new_points)
    
    for i in range(len(new_points)):
        df_new = pd.DataFrame({'type':'grid','ocgt':new_points[i,0],'wind':new_points[i,1],'solar':new_points[i,2],'feasable':True},index=[1])
        df_points = pd.concat([df_points,df_new],sort=False).fillna(0)
        
        

    #new_points = (list(new_points)+list(hull.points))
    
    #hull = Delaunay(new_points,incremental=False)
    
    #hull.add_points(new_points,restart=False)

    return df_points
    
                    
    

In [14]:
def constraint_functionality(network, snapshots,  MGA_slack = 0.05, constraint_values = [-1,-1,-1]):
    import pyomo.environ as pyomo_env

    variables = [gen_p for gen_p in network.model.generator_p_nom]

    types = ['OCGT','wind','olar']

    #OCGT
    if constraint_values[0] >= 0 :
        gen_p_type = [gen_p  for gen_p in variables if gen_p[-4:]==types[0]]
        gas_constraint = sum([network.model.generator_p_nom[gen_p] for gen_p in gen_p_type])
        network.model.gas_constraint = pyomo_env.Constraint(expr=gas_constraint==constraint_values[0])
    #Wind
    if constraint_values[1] >= 0 :
        gen_p_type = [gen_p  for gen_p in variables if gen_p[-4:]==types[1]]
        wind_constraint = sum([network.model.generator_p_nom[gen_p] for gen_p in gen_p_type])
        network.model.wind_constraint = pyomo_env.Constraint(expr=wind_constraint==constraint_values[1])
    #Solar
    if constraint_values[2] >= 0 :
        gen_p_type = [gen_p  for gen_p in variables if gen_p[-4:]==types[2]]
        solar_constraint = sum([network.model.generator_p_nom[gen_p] for gen_p in gen_p_type])
        network.model.solar_constraint = pyomo_env.Constraint(expr=solar_constraint==constraint_values[2])

In [15]:
def test_point(network,variables):

    out = network.lopf(network.snapshots,\
                 solver_name='gurobi',\
                 skip_pre=True,\
                 extra_functionality=lambda network,\
                 snapshots: constraint_functionality(network, snapshots, MGA_slack,variables),
                 solver_options = {'threads':8});

    feasable = str(network.results['Solver'].status) == 'ok' and (network.model.objective()<= (1 + MGA_slack) * old_objective_value)
      
    variables_out = [sum(network.generators.p_nom_opt[0:3]),sum(network.generators.p_nom_opt[3:6]),sum(network.generators.p_nom_opt[6:9])]
    variables_out_detailed = list(network.generators.p_nom_opt)
    
    return(feasable,variables_out_detailed)

In [16]:
def isBetween(a, b, c):
    # a Feasable point 
    # b infeasable point or feasable point if between test
    # c point to be tested
    
    between = True
    outside = False
    
    
    crossproduct = np.cross((b-a),(c-a))
    
    dotproduct = np.dot((c-a),(b-a))
                                
    # compare versus epsilon for floating point values, or != 0 if using integers
    # Test if c is on same axis as a and b
    #print(abs(crossproduct))
    epsilon = np.sqrt(abs(dotproduct))/10
    #print(epsilon)
    if any(abs(crossproduct) > epsilon):
        # c not in line with a and b
        between =  False
        outside = False
        return between, outside
    
    if all(a == b):
        between = False
        outside = False
        return between, outside      

    squaredlengthba = np.linalg.norm(b-a)**2
    #print(squaredlengthba)
    if dotproduct > squaredlengthba:
        between = False
        outside = False
        return between, outside
    
    #print(dotproduct)
    # Test if point c is further away from a then point b
    if dotproduct < 0:
        between =  False
        outside = True 
        return between, outside

    return between, outside
    

In [17]:
def reshape_grid(df_points):

    df_points['dist_to_mid']=np.sqrt(abs(df_points.ocgt - df_points[df_points.feasable].ocgt.mean())**2
                                    +abs(df_points.wind - df_points[df_points.feasable].wind.mean())**2
                                    +abs(df_points.solar - df_points[df_points.feasable].solar.mean())**2)

    df_points = df_points.sort_values(by=['dist_to_mid'])
    df_points = df_points.reset_index(drop=True)
    
    return df_points

### Creating grid of points  

In [18]:

dens = 4

gas, wind, solar  = np.meshgrid(np.linspace(df_points[df_points.type=='min ocgt'].ocgt,df_points[df_points.type=='max ocgt'].ocgt,dens),\
                                            np.linspace(df_points[df_points.type=='min wind'].wind,df_points[df_points.type=='max wind'].wind,dens),\
                                            np.linspace(df_points[df_points.type=='min solar'].solar,df_points[df_points.type=='max solar'].solar,dens))
gas = gas.reshape((dens)**3)
wind = wind.reshape((dens)**3)
solar = solar.reshape((dens)**3)

for i in range(len(gas)):
    df_points.loc[df_points.index.max()+1] = ['grid',False,gas[i],wind[i],solar[i]]

df_points = reshape_grid(df_points)


In [19]:
print('Computation time estimate: ' + str(len(df_points)*1.3/60) + 'min')

Computation time estimate: 1.5383333333333333min


### Computing feasable points

In [20]:
from scipy.spatial import ConvexHull, convex_hull_plot_2d, Delaunay


dhull = Delaunay(df_points[df_points.feasable][['ocgt','wind','solar']],incremental=True)


simulations = 0

df_points = fill_hull(df_points,dhull,10)


In [21]:

t = time.time()

for index, row in df_points[df_points.feasable==False].iterrows():
    
    point3 = [row.ocgt,row.wind,row.solar]
    
    feasable = df_points[df_points.feasable][['ocgt','wind','solar']].values
    infeasable = df_points[df_points.feasable==False][['ocgt','wind','solar']].values
    
    #between = any([isBetween(point1,point2,point3)[0] for point1 in grid.feasable for point2 in grid.feasable])
    outside = any([isBetween(point2,point1,point3)[1] for point1 in feasable for point2 in infeasable])
    inside = in_hull(p=point3,hull=dhull)
    #print(between,outside,inside)
    
    if (not inside) and (not outside) :
        is_good , variables_out = test_point(network,point3)
        simulations += 1
        if is_good:
            df_points.iloc[index].feasable = True
            df_detail.loc[df_detail.index.max()+1]=list(network.generators.p_nom_opt.values)
            #grid.feasable.append(point3)
            dhull.add_points([point3],True)
            #print(grid.feasable)
            #grid.detailed.append(variables_out)
        #else : 
            #grid.infeasable.append(point3)
    elif inside:
        dhull.add_points([point3],False)
        df_points.iloc[index].feasable = True
    #else :
        #grid.irelevant.append(point3)
        #df_points.type = 'irrelevant'
elapsed = time.time() - t


INFO:pypsa.opf:Building pyomo model using `angles` formulation
Index(['gas 0', 'gas 1', 'gas 2', 'wind 0', 'wind 1', 'wind 2', 'solar 0',
       'solar 1', 'solar 2'],
      dtype='object', name='name')
Currently PyPSA cannot do both these functions, so PyPSA is choosing investment optimisation for these generators.
INFO:pypsa.opf:Solving model using gurobi
INFO:pypsa.opf:Optimization successful


# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: x655
  Lower bound: 17780711363.30111
  Upper bound: 17780711363.30111
  Number of objectives: 1
  Number of constraints: 922
  Number of variables: 655
  Number of binary variables: 0
  Number of integer variables: 0
  Number of continuous variables: 655
  Number of nonzeros: 1939
  Sense: minimize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  Return code: 0
  Message: Model was solved to optimality (subject to tolerances), and an optimal solution is available.
  Termination condition: optimal
  Termination message: Model w

INFO:pypsa.opf:Building pyomo model using `angles` formulation
Index(['gas 0', 'gas 1', 'gas 2', 'wind 0', 'wind 1', 'wind 2', 'solar 0',
       'solar 1', 'solar 2'],
      dtype='object', name='name')
Currently PyPSA cannot do both these functions, so PyPSA is choosing investment optimisation for these generators.
INFO:pypsa.opf:Solving model using gurobi
    message from solver=Problem proven to be infeasible or unbounded.
ERROR:pypsa.opf:Optimisation failed with status warning and terminal condition infeasibleOrUnbounded


    model=Linear Optimal Power Flow;
        message from solver=Problem proven to be infeasible or unbounded.
# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: x655
  Lower bound: -inf
  Upper bound: inf
  Number of objectives: 1
  Number of constraints: 922
  Number of variables: 655
  Number of binary variables: 0
  Number of integer variables: 0
  Number of continuous variables: 655
  Number of nonzeros: 1939
  Sense: minimize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: warning
  Return code: 0
  Message: Problem proven to be infeasible or unbounded.
  Termination conditio

INFO:pypsa.opf:Building pyomo model using `angles` formulation
Index(['gas 0', 'gas 1', 'gas 2', 'wind 0', 'wind 1', 'wind 2', 'solar 0',
       'solar 1', 'solar 2'],
      dtype='object', name='name')
Currently PyPSA cannot do both these functions, so PyPSA is choosing investment optimisation for these generators.
INFO:pypsa.opf:Solving model using gurobi
    message from solver=Problem proven to be infeasible or unbounded.
ERROR:pypsa.opf:Optimisation failed with status warning and terminal condition infeasibleOrUnbounded


    model=Linear Optimal Power Flow;
        message from solver=Problem proven to be infeasible or unbounded.
# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: x655
  Lower bound: -inf
  Upper bound: inf
  Number of objectives: 1
  Number of constraints: 922
  Number of variables: 655
  Number of binary variables: 0
  Number of integer variables: 0
  Number of continuous variables: 655
  Number of nonzeros: 1939
  Sense: minimize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: warning
  Return code: 0
  Message: Problem proven to be infeasible or unbounded.
  Termination conditio

INFO:pypsa.opf:Building pyomo model using `angles` formulation
Index(['gas 0', 'gas 1', 'gas 2', 'wind 0', 'wind 1', 'wind 2', 'solar 0',
       'solar 1', 'solar 2'],
      dtype='object', name='name')
Currently PyPSA cannot do both these functions, so PyPSA is choosing investment optimisation for these generators.
INFO:pypsa.opf:Solving model using gurobi
INFO:pypsa.opf:Optimization successful


# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: x655
  Lower bound: 18609758790.938496
  Upper bound: 18609758790.938496
  Number of objectives: 1
  Number of constraints: 922
  Number of variables: 655
  Number of binary variables: 0
  Number of integer variables: 0
  Number of continuous variables: 655
  Number of nonzeros: 1939
  Sense: minimize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  Return code: 0
  Message: Model was solved to optimality (subject to tolerances), and an optimal solution is available.
  Termination condition: optimal
  Termination message: Model

In [22]:
print(elapsed/60)
print(simulations)
elapsed/60/simulations

0.44026983976364137
4


0.11006745994091034

In [23]:
#np.save('feasable_points_euro30',np.array(list(grid.feasable) + max_min_points + [original_solution]))

In [24]:
#grid.feasable = np.load('feasable_points.npy')

### Plotting feasable region

In [28]:
df_points.type

0          grid
1          grid
2      original
3          grid
4          grid
5          grid
6          grid
7     min solar
8          grid
9          grid
10         grid
11         grid
12         grid
13         grid
14         grid
15         grid
16         grid
17         grid
18         grid
19         grid
20         grid
21         grid
22         grid
23         grid
24         grid
25         grid
26         grid
27    max solar
28         grid
29         grid
        ...    
1          grid
1          grid
1          grid
1          grid
1          grid
1          grid
1          grid
1          grid
1          grid
1          grid
1          grid
1          grid
1          grid
1          grid
1          grid
1          grid
1          grid
1          grid
1          grid
1          grid
1          grid
1          grid
1          grid
1          grid
1          grid
1          grid
1          grid
1          grid
1          grid
1          grid
Name: type, Length: 160,

In [44]:
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

#for i in range(len(max_min_points)):
#       ax.plot([max_min_points[i][0]],[max_min_points[i][1]],[max_min_points[i][2]], "go",markersize=10)


ax.plot(df_points[df_points.feasable]['ocgt'],df_points[df_points.feasable]['wind'],df_points[df_points.feasable]['solar'], "bo",markersize=7)
ax.plot(df_points[df_points.feasable==False]['ocgt'],df_points[df_points.feasable==False]['wind'],df_points[df_points.feasable==False]['solar'], "ro",markersize=7)
ax.plot(df_points[df_points.type=='original']['ocgt'],df_points[df_points.type=='original']['wind'],df_points[df_points.type=='original']['solar'], "co",markersize=7)

#ax.plot(grid.irelevant[:,0],grid.irelevant[:,1],grid.irelevant[:,2], "co",markersize=6)
#ax.plot([original_solution[0]],[original_solution[1]],[original_solution[2]],'yo')
    
plt.show();
ax.set_xlabel('Gas');
ax.set_ylabel('Wind');
ax.set_zlabel('Solar');

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [45]:
hull = ConvexHull(dhull.points)
hull.volume

2266922180876.639

In [52]:
df_detail.corr().head()

name,gas 0,gas 1,gas 2,wind 0,wind 1,wind 2,solar 0,solar 1,solar 2,AT onwind,...,SE3 onwind,SE offwind,SE solar,SE OCGT,SI onwind,SI solar,SI OCGT,SK onwind,SK solar,SK OCGT
name,,,,,,,,,,,,,,,,,,,,,
gas 0,1.000000,1.000000,1.000000,0.983308,0.983308,0.983308,0.998981,0.998981,0.998981,0.494086,...,0.494086,0.494086,NaN,-0.197881,0.494086,NaN,0.354040,0.494086,NaN,0.368069
gas 1,1.000000,1.000000,1.000000,0.983308,0.983308,0.983308,0.998981,0.998981,0.998981,0.494086,...,0.494086,0.494086,NaN,-0.197881,0.494086,NaN,0.354040,0.494086,NaN,0.368069
gas 2,1.000000,1.000000,1.000000,0.983308,0.983308,0.983308,0.998981,0.998981,0.998981,0.494086,...,0.494086,0.494086,NaN,-0.197881,0.494086,NaN,0.354040,0.494086,NaN,0.368069
wind 0,0.983308,0.983308,0.983308,1.000000,1.000000,1.000000,0.990519,0.990519,0.990519,0.644028,...,0.644028,0.644028,NaN,-0.207272,0.644028,NaN,0.356305,0.644028,NaN,0.369823
wind 1,0.983308,0.983308,0.983308,1.000000,1.000000,1.000000,0.990519,0.990519,0.990519,0.644028,...,0.644028,0.644028,NaN,-0.207272,0.644028,NaN,0.356305,0.644028,NaN,0.369823


In [49]:
import matplotlib.pyplot as plt

plt.matshow(df_detail.corr())
plt.show()
cb = plt.colorbar()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …